# Notebook for visualizing MVP phase 2 coherence check results

Coherence check includes results generated by FastQC, Samtools Flagstat, and RTG Vcfstats.

In [ ]:
import StringIO

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import google.datalab.bigquery as bq

# MVP Phase 2 FastQC Results (n=1902)

In [ ]:
mvp_fastqc_table=""

## Per base quality

In [ ]:
per_base_qual = bq.Query('SELECT index, ' +
                         'ROUND(CAST(value AS FLOAT64), 1) AS value_tenth, ' +
                         'COUNT(*) AS observations ' +
                         'FROM `' + mvp_fastqc_table + '` ' +
                         'WHERE dimension = "per_base_qual" ' +
                         'GROUP BY index, value_tenth').execute().result().to_dataframe()
per_base_qual.head()

In [ ]:
start_end = pd.DataFrame(per_base_qual['index'].str.split('-',1).tolist(), columns=['start', 'end'])
per_base_qual = per_base_qual.join(start_end)

per_base_qual[['value_tenth']] = per_base_qual[['value_tenth']].apply(pd.to_numeric)
per_base_qual[['start']] = per_base_qual[['start']].apply(pd.to_numeric)
per_base_qual[['observations']] = per_base_qual[['observations']].apply(pd.to_numeric)

per_base_qual.head()

In [ ]:
per_base_qual[['log_observations']] = np.log(per_base_qual[['observations']])
per_base_qual.head()

In [ ]:
plot = per_base_qual.plot(kind='scatter', x='start', y='value_tenth', c='log_observations', s=35)

In [ ]:
per_base_qual.plot.hexbin(x='start', y='value_tenth', C='log_observations', reduce_C_function=np.max, gridsize=50)
plt.show()

In [ ]:
fig, ax = plt.subplots()
scat = per_base_qual.plot(kind='scatter', x='start', y='value_tenth', c='log_observations', cmap='plasma', ax=ax)
ax.set_xlim(0,140)
ax.set_title("Distribution of Per-base Sequence Quality across Genomes")
ax.set_xlabel("Base position")
ax.set_ylabel("Quality score")
plt.show()

## GC Content

In [ ]:
dimension = "gc_content"
gc_content = bq.Query('SELECT sample, ' +
                      'dimension, ' +
                      'CAST(value AS INT64) as value ' +
                      'FROM `' + mvp_fastqc_table + '` ' +
                      'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
gc_content.head()

In [ ]:
gc_content_counts = gc_content.groupby('value')['sample'].nunique()
print gc_content_counts

In [ ]:
fig, ax = plt.subplots()
bar = gc_content_counts.plot.bar(alpha=0.5)
ax.set_title("Distribution of Reads GC Content by Genome")
ax.set_xlabel("GC Content (%)")
ax.set_ylabel("Frequency")
plt.show()

## Average sequence quality

In [ ]:
dimension = "seq_quality"
avg_seq_qual = bq.Query('SELECT sample, ' +
                        'SUM(CAST(index as INT64) * CAST(value AS FLOAT64))/SUM(CAST(value AS FLOAT64)) ' +
                        'FROM `' + mvp_fastqc_table + '` ' +
                        'WHERE dimension = "' + dimension + '" ' +
                        'GROUP BY sample').execute().result().to_dataframe()
avg_seq_qual.head()

In [ ]:
plt.figure()
plt.boxplot(avg_seq_qual['f0_'], 0, 'gD')
plt.show()

In [ ]:
fig, ax = plt.subplots()
avg_seq_qual.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Average Read Qualities")
ax.set_xlabel("Average Read Quality")
ax.set_ylabel("Frequency")
plt.show()

## Sequence length

In [ ]:
dimension = "seq_len"
seq_len = bq.Query('SELECT sample, dimension, ' +
                   'CAST(value AS INT64) as value ' + 
                   'FROM `' + mvp_fastqc_table + '` ' + 
                   'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
seq_len.head()

In [ ]:
seq_len_counts = seq_len.groupby('value')['sample'].nunique()
print seq_len_counts

In [ ]:
fig, ax = plt.subplots()
bar = seq_len_counts.plot.bar(alpha=0.5)
ax.set_title("Distribution of Read Lengths")
ax.set_xlabel("Read Length")
ax.set_ylabel("Frequency")
plt.show()

# MVP Phase 2 VCFStats Results (n=1902)

In [ ]:
mvp_vcfstats_table = ""

## SNPs per genome

In [ ]:
snps = bq.Query('SELECT sample, dimension, ' +
                'CAST(value AS INT64) as value ' +
                'FROM `' + mvp_vcfstats_table + '` ' +
                'WHERE dimension = "snps"').execute().result().to_dataframe()
snps.head()

In [ ]:
fig, ax = plt.subplots()
hist = snps.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of SNPs per Genome")
ax.set_xlabel("SNP Count")
ax.set_ylabel("Frequency")
plt.show()

## Check number of variants in each category
I am not sure whether RTG Tools vcfstats considers failed variants, and their documentation of vcfstats is very poor. Eye-check of ~10 vcfstats files does not show any variants marked as "Failed Filters".

I'm going to run a query to see if any were marked as failing.

In [ ]:
dimension_sums = bq.Query('SELECT dimension, SUM(CAST(value AS INT64)) AS sum ' +
                'FROM `' + mvp_vcfstats_table + '` ' +
                'WHERE dimension IN ("failed_filters", "passed_filters") ' +
                'GROUP BY dimension').execute().result().to_dataframe()
dimension_sums.head()

## Indels per genome

In [ ]:
dimension = "indels"
indels = bq.Query('SELECT sample, dimension, ' +
                  'CAST(value AS INT64) as value ' +
                  'FROM `' + mvp_vcfstats_table + '` ' +
                  'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
indels.head()

In [ ]:
fig, ax = plt.subplots()
hist = indels.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of INDELs per Genome")
ax.set_xlabel("INDEL Count")
ax.set_ylabel("Frequency")
plt.show()

In [ ]:
box = indels.plot.box()

In [ ]:
plt.figure()
plt.boxplot(indels['value'], 0, 'gD')
plt.show()

## Cumulative insertions, deletions, and indels
RTG Tools vcfstats outputs data on three categories of indels: insertions, deletions, and indels. From this google group post (https://groups.google.com/a/realtimegenomics.com/forum/#!searchin/rtg-users/vcfstats/rtg-users/-eFsSbWF6ks/1HrnevHTAgAJ):

>For the insertions/deletions/indels the table is based on the delta in length rather than total length (which really matters for the indels):

>Insertions (pure addition of bases)
>A -> AT (length 1 insertion)
>ATT -> ATTTT (length 2 insertion) 

>Deletions (pure deletion of bases)
>AT -> A (length 1 deletion)
>ATTTT -> ATT (length 2 deletion)

>Indels (length changing but not pure)
>ATT -> CTTT (length 1 indel)
>CTTT -> ATT (length 1 indel)"

In [ ]:
indels = bq.Query('SELECT sample, SUM(DISTINCT CAST(value AS INT64)) AS cum_indels ' +
                  'FROM `' + mvp_vcfstats_table + '` ' +
                  'WHERE dimension = "insertions" ' +
                  'OR dimension = "deletions" ' +
                  'OR dimension = "indels" ' +
                  'GROUP BY sample').execute().result().to_dataframe()
indels.head()

In [ ]:
fig, ax = plt.subplots()
hist = indels.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of INDELs per Genome")
ax.set_xlabel("INDEL Count")
ax.set_ylabel("Frequency")
plt.show()

## Frequency of Ti/Tv ratios

In [ ]:
dimension = "ti_tv_ratio"
ti_tv = bq.Query('SELECT sample, dimension, ' + 
                 'CAST(value AS FLOAT64) as value ' + 
                 'FROM `' + mvp_vcfstats_table + '` ' + 
                 'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
ti_tv.head()

In [ ]:
fig, ax = plt.subplots()
hist = ti_tv.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Ti/Tv ratios")
ax.set_xlabel("Ti/Tv Ratio")
ax.set_ylabel("Frequency")
plt.show()

In [ ]:
plt.figure()
plt.boxplot(ti_tv['value'], 0, 'gD')
plt.show()

## Frequency of SNP het/hom ratios

In [ ]:
snp_het_hom_ratio = bq.Query('SELECT sample, dimension, ' + 
                             'CAST(value AS FLOAT64) as value ' + 
                             'FROM `' + mvp_vcfstats_table + '` ' + 
                             'WHERE dimension = "snp_het_hom_ratio"').execute().result().to_dataframe()
snp_het_hom_ratio.head()

In [ ]:
fig, ax = plt.subplots()
hist = snp_het_hom_ratio.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of SNP het/hom ratios")
ax.set_xlabel("Het/Hom Ratio")
ax.set_ylabel("Frequency")
plt.show()

# MVP Phase 2 Samtools Flagstat Results (n=1902)

In [ ]:
mvp_flagstat_table = ""

## Percent of reads mapped to reference

In [ ]:
dimension = "mapped_reads_perc"
mapped_reads_perc = bq.Query('SELECT sample, dimension, ' + 
                             'CAST(value AS FLOAT64) as value ' + 
                             'FROM `' + mvp_flagstat_table + '` ' + 
                             'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
mapped_reads_perc.head()

In [ ]:
fig, ax = plt.subplots()
hist = mapped_reads_perc.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Mapped Reads (%)")
ax.set_xlabel("Percent Mapped Reads")
ax.set_ylabel("Frequency")
plt.show()

## Count of reads mapped to reference

In [ ]:
dimension = "mapped_reads_count"
mapped_reads_count = bq.Query('SELECT sample, dimension, ' + 
                              'CAST(value AS FLOAT64) as value ' + 
                              'FROM `' + mvp_flagstat_table + '` ' + 
                              'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
mapped_reads_count.head()

In [ ]:
fig, ax = plt.subplots()
hist = mapped_reads_count.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Mapped Reads (Count)")
ax.set_xlabel("Count of Mapped Reads (Billions)")
ax.set_ylabel("Frequency")
plt.show()

## Percent of reads properly paired

In [ ]:
dimension = "properly_paired_perc"
properly_paired_perc = bq.Query('SELECT sample, dimension, ' + 
                                'CAST(value AS FLOAT64) as value ' + 
                                'FROM `' + mvp_flagstat_table + '` ' + 
                                'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
properly_paired_perc.head()

In [ ]:
fig, ax = plt.subplots()
hist = properly_paired_perc.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Properly Paired Reads (%)")
ax.set_xlabel("Percent Properly Paired Reads")
ax.set_ylabel("Frequency")
plt.show()

## Count of reads properly paired

In [ ]:
dimension = "properly_paired_count"
properly_paired_count = bq.Query('SELECT sample, dimension, ' + 
                                 'CAST(value AS FLOAT64) as value ' + 
                                 'FROM `' + mvp_flagstat_table + '` ' + 
                                 'WHERE dimension = "' + dimension + '"').execute().result().to_dataframe()
properly_paired_count.head()

In [ ]:
fig, ax = plt.subplots()
hist = properly_paired_count.hist(alpha=0.5, ax=ax)
ax.set_title("Distribution of Properly Paired Reads (Count)")
ax.set_xlabel("Count of Properly Paired Reads (Billions)")
ax.set_ylabel("Frequency")
plt.show()